In [1]:
import torch
import torchvision


print(torchvision.__version__)
print(torch.__version__)

0.21.0
2.6.0


In [8]:
try:
    import going_modular
    
except: 
    print("not found")
    !git clone git@github.com:mrdbourke/pytorch-deep-learning.git
    !mv pytorch-deep-learning/going_modular . 
    !rm -rf pytorch-deep-learning


not found
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 4393, done.
remote: Counting objects: 100% (1532/1532), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 4393 (delta 1456), reused 1401 (delta 1401), pack-reused 2861 (from 3)
Receiving objects: 100% (4393/4393), 649.93 MiB | 8.41 MiB/s, done.
Resolving deltas: 100% (2659/2659), done.
Updating files: 100% (248/248), done.


In [10]:
from going_modular.going_modular import engine, data_setup

device = "cuda" if torch.cuda.is_available() else "cpu"

device

'cpu'

In [26]:
# Get data
import os
import zipfile
from pathlib import Path
import requests

# Setup data path
data_path = Path("data/")
image_path = data_path/"pizza_steak_sushi"

image_path

# If image folder doesnt exist download

if image_path.is_dir():
    print("Path exist... skipping downlaod")
else:
    print("make dir and Download the file")
    image_path.mkdir(parents=True, exist_ok=True)
    
    # download the zip and write it down
    
    with open("data/pizza_steak_sushi.zip", "wb") as f:
        req = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        f.write(req.content)
    

# Now unzip the file
with  zipfile.ZipFile(data_path/"pizza_steak_sushi.zip", "r") as f:
    f.extractall(image_path)
    os.remove(data_path/"pizza_steak_sushi.zip")
    
    



Path exist... skipping downlaod


In [22]:
data_path,  image_path

(PosixPath('data'), PosixPath('data/pizza_steak_sushi'))

In [30]:
# setup the directory path

train_dir = image_path/"train"
test_dir = image_path/"test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [35]:
# convert the dir data into dataloaders
from going_modular.going_modular import data_setup
from torchvision import transforms

# manual transfrom 
normalize  =  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
manual_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor(),
    normalize
])
BATCH_SIZE = 32
train_data_loader, test_data_loader, class_names =  data_setup.create_dataloaders(train_dir,test_dir,manual_transform, BATCH_SIZE, 1)

In [40]:
# Automatic transform
from torchvision.models import EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.DEFAULT

auto_transfrom = weights.transforms()

auto_transfrom

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [41]:
BATCH_SIZE = 32
train_data_loader, test_data_loader, class_names =  data_setup.create_dataloaders(train_dir,test_dir,auto_transfrom, BATCH_SIZE, 1)

In [43]:
# Getting a pre-trained model
weights = EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [44]:
from torchinfo import summary

summary(model=model)

Layer (type:depth-idx)                                  Param #
EfficientNet                                            --
├─Sequential: 1-1                                       --
│    └─Conv2dNormActivation: 2-1                        --
│    │    └─Conv2d: 3-1                                 864
│    │    └─BatchNorm2d: 3-2                            64
│    │    └─SiLU: 3-3                                   --
│    └─Sequential: 2-2                                  --
│    │    └─MBConv: 3-4                                 1,448
│    └─Sequential: 2-3                                  --
│    │    └─MBConv: 3-5                                 6,004
│    │    └─MBConv: 3-6                                 10,710
│    └─Sequential: 2-4                                  --
│    │    └─MBConv: 3-7                                 15,350
│    │    └─MBConv: 3-8                                 31,290
│    └─Sequential: 2-5                                  --
│    │    └─MBConv: 3-9         